In [6]:
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Convolution2D, Flatten, Dense

In [7]:
# Reading the csv file and converting it into an array
data=pd.read_csv('/content/drive/MyDrive/Processed_Data.csv')
data=data.drop('Unnamed: 0',axis=1)
data=data.values
imgs=[]
steering=[]
for i in range(len(data)):
    imgs.append('/content/drive/MyDrive/IMG/' + data[i][0].split('\\')[-1])
    steering.append(data[i][3])

In [8]:
imgs=np.array(imgs)
steering=np.array(steering)

In [9]:
# Spliting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(imgs, steering, test_size = 0.2)

In [10]:

def batch_generator(imgs, steering, batch_size):   
   
    while True:
        
        X = []
        y = []
    
        for i in range(batch_size):
        
            index = np.random.randint(0,len(imgs)-1)
            img = cv.imread(imgs[index])
            
                
            img = cv.cvtColor(img, cv.COLOR_BGR2YUV)
            img = img[60:135,:,:]
            
            img = cv.resize(img , (200,66))
            
            

            if (np.random.rand() > 0.5):

                brighness_cont = np.random.randint(1,4)
                img = img / brighness_cont

            if (np.random.rand() > 0.5):

                img = cv.flip(img, 1)
                steering_angle = -float(steering[index])

            else:
                steering_angle = float(steering[index])
            
            
            X.append(img)
            y.append(steering_angle)
            
        X = np.array(X)
        y = np.array(y)

        yield X,y

In [11]:
model = Sequential()

model.add(Convolution2D(24, (5,5), strides = (2,2), input_shape = (66,200,3), activation = 'elu'))
model.add(Convolution2D(36, (5,5), strides = (2,2), activation = 'elu'))
model.add(Convolution2D(48, (5,5), strides = (2,2), activation = 'elu'))

model.add(Convolution2D(64, (3,3), activation = 'elu'))
model.add(Convolution2D(64, (3,3), activation = 'elu'))

model.add(Flatten())

model.add(Dense(100, activation = 'elu'))
model.add(Dense(50, activation = 'elu'))
model.add(Dense(10, activation = 'elu'))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               1

In [13]:
history = model.fit(batch_generator(X_train, y_train, 64), steps_per_epoch = 300, epochs = 8,
         validation_data = batch_generator(X_test,y_test, 64), validation_steps = 200)

Epoch 1/8
300/300 [==============================] - 139s 456ms/step - loss: 5.3599 - accuracy: 0.2925 - val_loss: 0.0931 - val_accuracy: 0.3215
Epoch 2/8
300/300 [==============================] - 137s 459ms/step - loss: 0.0996 - accuracy: 0.3254 - val_loss: 0.0911 - val_accuracy: 0.3321
Epoch 3/8
300/300 [==============================] - 138s 462ms/step - loss: 0.0891 - accuracy: 0.3099 - val_loss: 0.0800 - val_accuracy: 0.3227
Epoch 4/8
300/300 [==============================] - 139s 466ms/step - loss: 0.0778 - accuracy: 0.3217 - val_loss: 0.0816 - val_accuracy: 0.3168
Epoch 5/8
300/300 [==============================] - 138s 463ms/step - loss: 0.0697 - accuracy: 0.3242 - val_loss: 0.0840 - val_accuracy: 0.3181
Epoch 6/8
300/300 [==============================] - 138s 461ms/step - loss: 0.0617 - accuracy: 0.3132 - val_loss: 0.0848 - val_accuracy: 0.3221
Epoch 7/8
300/300 [==============================] - 137s 457ms/step - loss: 0.0532 - accuracy: 0.3301 - val_loss: 0.0886 - val_ac

In [14]:
model.save('SDC.h5')

In [15]:
model=load_model('SDC.h5')